In [1]:
!pip install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923

  Cloning https://github.com/rongardF/tvdatafeed.git (to revision e6f6aaa7de439ac6e454d9b26d2760ded8dc4923) to c:\users\wangq\appdata\local\temp\pip-req-build-_lziznbo
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git 'C:\Users\wangq\AppData\Local\Temp\pip-req-build-_lziznbo'
  Running command git rev-parse -q --verify 'sha^e6f6aaa7de439ac6e454d9b26d2760ded8dc4923'
  Running command git fetch -q https://github.com/rongardF/tvdatafeed.git e6f6aaa7de439ac6e454d9b26d2760ded8dc4923


In [8]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime

tv = TvDatafeed()

In [11]:
# futures NQ and ES as list
futures = ["NQ1!", "ES1!"]
# periods enum and short string list
periods = [[Interval.in_1_minute, "1M"],
            [Interval.in_5_minute, "5M"],
            [Interval.in_30_minute, "30M"],
            [Interval.in_4_hour, "4H"],
            [Interval.in_daily,"1D"]
            ]
# print(periods[3][1])
periods[1]

[<Interval.in_5_minute: '5'>, '5M']

In [4]:
# this design make it more flexible to add more symbol and interval.
# merge them to new list
merged = [[future] + period for future in futures for period in periods]
print(merged)

[['NQ1!', <Interval.in_1_minute: '1'>, '1M'], ['NQ1!', <Interval.in_5_minute: '5'>, '5M'], ['ES1!', <Interval.in_1_minute: '1'>, '1M'], ['ES1!', <Interval.in_5_minute: '5'>, '5M']]


In [5]:
df = tv.get_hist(symbol='NQ1!', exchange='CME_MINI', interval=Interval.in_1_minute, n_bars=100000)
print(df)

                            symbol      open      high       low     close  \
datetime                                                                     
2024-10-13 17:00:00  CME_MINI:NQ1!  20432.50  20440.00  20423.25  20428.00   
2024-10-13 17:01:00  CME_MINI:NQ1!  20427.00  20427.00  20418.00  20420.75   
2024-10-13 17:02:00  CME_MINI:NQ1!  20420.75  20422.75  20415.75  20420.50   
2024-10-13 17:03:00  CME_MINI:NQ1!  20421.25  20425.25  20407.50  20413.25   
2024-10-13 17:04:00  CME_MINI:NQ1!  20414.00  20420.00  20414.00  20419.50   
...                            ...       ...       ...       ...       ...   
2024-10-23 22:02:00  CME_MINI:NQ1!  20335.00  20335.00  20333.50  20333.50   
2024-10-23 22:03:00  CME_MINI:NQ1!  20335.00  20335.75  20334.00  20334.75   
2024-10-23 22:04:00  CME_MINI:NQ1!  20335.00  20341.00  20334.75  20336.25   
2024-10-23 22:05:00  CME_MINI:NQ1!  20337.00  20337.50  20335.25  20335.25   
2024-10-23 22:06:00  CME_MINI:NQ1!  20334.25  20336.00  20332.00

In [6]:
import pandas as pd
from sqlalchemy import create_engine, inspect

engine = create_engine('sqlite:///data/ESNQ_DB.db')

In [7]:
table_names = inspect(engine).get_table_names()

print("Tables in the database:")
table_names

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
files = ['NQ_2024-08-29_2024-09-13_1M.csv',
        'NQ_2024-08-18_2024-09-13_5M.csv',
        'NQ_2024-01-01_2024-09-13_30M.csv',
        'NQ_2021-01-03_2024-09-13_4H.csv',
        'NQ_1999-06-29_2024-09-12_1D.csv',
        'ES_2024-08-29_2024-09-13_1M.csv',
        'ES_2024-08-18_2024-09-13_5M.csv',
        'ES_2022-10-18_2024-09-13_30M.csv',
        'ES_2021-01-03_2024-09-13_4H.csv',
        'ES_1997-09-08_2024-09-12_1D.csv']

files


In [ ]:
for file in files:
    print(file)


In [19]:
def sql_importer (df, table_name):
    # Use inspector to check if the table exists
    inspector = inspect(engine)
    if not inspector.has_table(table_name):
        df.to_sql(table_name, engine, if_exists='append')
        print (f'New table created for {table_name} with {str(len(df))} rows')
    else:
        max_date = pd.read_sql(f'SELECT MAX(datetime) FROM {table_name}', engine).values[0][0]
        print(max_date)
        df = df[df.datetime > max_date]
        df.to_sql(table_name, engine, if_exists='append')
        print(str(len(df)) + ' new rows imported to DB')

In [ ]:
# Process the array to extract instrument and time frame
tablenames = [f"{file.split('_')[0]}{file.split('_')[-1].replace('.csv', '')}" for file in files]
tablenames

In [17]:
import pandas as pd
from sqlalchemy import create_engine
# merged = zip(files, tablenames)
merged = list(zip(files, tablenames))


In [20]:
import os
datadir = 'stockdata'
for file, tablename in merged:
    print(f"Loading data into {tablename}")
    filename = os.path.join(datadir, file)
    df = pd.read_csv(filename)
    sql_importer(df, tablename)

Loading data into NQ1M
2024-09-06 15:59:00
6899 new rows imported to DB
Loading data into NQ5M
2024-09-06 15:55:00
1380 new rows imported to DB
Loading data into NQ30M
2024-09-06 15:30:00
230 new rows imported to DB
Loading data into NQ4H
2024-09-06 13:00:00
30 new rows imported to DB
Loading data into NQ1D
2024-09-05 17:00:00
5 new rows imported to DB
Loading data into ES1M
2024-09-06 15:59:00
6900 new rows imported to DB
Loading data into ES5M
2024-09-06 15:55:00
1380 new rows imported to DB
Loading data into ES30M
2024-09-06 15:30:00
230 new rows imported to DB
Loading data into ES4H
2024-09-06 13:00:00
30 new rows imported to DB
Loading data into ES1D
2024-09-05 17:00:00
5 new rows imported to DB


In [21]:

# read from database and test
pd.read_sql('ES1M', engine)

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-06-02 17:00:00,CME_MINI:NQ1!,18590.25,18598.50,18576.00,18595.25,876.0
1,1,2024-06-02 17:01:00,CME_MINI:NQ1!,18594.25,18597.00,18587.75,18587.75,350.0
2,2,2024-06-02 17:02:00,CME_MINI:NQ1!,18588.50,18588.50,18562.25,18564.50,594.0
3,3,2024-06-02 17:03:00,CME_MINI:NQ1!,18563.75,18567.50,18557.25,18560.00,573.0
4,4,2024-06-02 17:04:00,CME_MINI:NQ1!,18560.50,18564.50,18553.75,18553.75,209.0
...,...,...,...,...,...,...,...,...
89941,15075,2024-09-13 15:55:00,CME_MINI:ES1!,5627.00,5627.25,5626.25,5626.25,247.0
89942,15076,2024-09-13 15:56:00,CME_MINI:ES1!,5626.25,5626.75,5626.25,5626.50,79.0
89943,15077,2024-09-13 15:57:00,CME_MINI:ES1!,5626.75,5627.00,5626.75,5626.75,62.0
89944,15078,2024-09-13 15:58:00,CME_MINI:ES1!,5627.00,5627.50,5626.75,5627.25,279.0
